In [ ]:
# Loading DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load your EEG data
df = pd.read_csv('features_raw.csv')
print(f"Data shape: {df.shape}")
print(f"Channels: {df.columns.tolist()}")

Data shape: (8064, 33)
Channels: ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'Unnamed: 32']


In [ ]:
# FRONTAL CHANNELS:
# FP1 - Left frontopolar
# FP2 - Right frontopolar
# AF3 - Left anterior frontal
# AF4 - Right anterior frontal
# F3 - Left frontal
# F4 - Right frontal
# F7 - Left lateral frontal
# F8 - Right lateral frontal
# Fz - Midline frontal

# FRONTOCENTRAL CHANNELS:
# FC5 - Left frontocentral
# FC6 - Right frontocentral
# FC1 - Left frontocentral
# FC2 - Right frontocentral

# CENTRAL CHANNELS:
# C3 - Left central
# C4 - Right central
# Cz - Midline central
# T7 - Left temporal
# T8 - Right temporal

# CENTROPARIETAL CHANNELS:
# CP5 - Left centroparietal
# CP6 - Right centrparietal
# CP1 - Left centroparietal
# CP2 - Right centroparietal

# PARIETAL CHANNELS:
# P3 - Left parietal
# P4 - Right parietal
# P7 - Left parietal
# P8 - Right parietal
# Pz - Midline parietal

# PARIETO-OCCIPTAL CHANNELS:
# PO3 - Left parieto-occipital
# PO4 - Right parieto-occipital

# OCCIPITAL CHANNELS:
# O1 - Left occipital
# O2 - Right occipital
# Oz - Midline occipital